In [1]:
!pip install tabula-py jpype1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 33.4 MB/s eta 0:00:00


In [7]:
import numpy as np
import tabula
import pandas as pd

In [19]:
# !curl -O https://bocongan.gov.vn/KND/TT/Lists/TinTuc/Attachments/40705/Danh%20sach%20hoc%20vien.pdf


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0


In [8]:
dfs = tabula.read_pdf("./danh_sach_hoc_vien.pdf", pages="all", pandas_options= {'dtype': str}, multiple_tables=True)

In [20]:
dfs[900]

,TT,Họ và tên,Giới\rtính,Quốc tịch,Hạng,Số GPLX,Mã khóa học,Ngày khai\rgiảng,Ngày bế\rgiảng
0,62062,NGUYỄN XUÂN ĐƯỜNG,Nam,Việt Nam,B2,NaN,75023K22B2196,30/11/2022,04/03/2023
1,62063,NGÔ QUANG DUY,Nam,Việt Nam,B2,NaN,75023K22B2196,30/11/2022,04/03/2023
2,62064,BÙI T.TRƯỜNG GIANG,Nam,Việt Nam,B2,NaN,75023K22B2196,30/11/2022,04/03/2023
3,62065,NGUYỄN TIẾN HÀ,Nam,Việt Nam,B2,NaN,75023K22B2196,30/11/2022,04/03/2023
4,62066,PHAN THỊ HÀ,Nữ,Việt Nam,B2,NaN,75023K22B2196,30/11/2022,04/03/2023
...,...,...,...,...,...,...,...,...,...
64,62126,ĐÀO VĂN TRÍ,Nam,Việt Nam,B2,NaN,75023K22B2196,30/11/2022,04/03/2023
65,62127,HOÀNG MINH TRUNG,Nam,Việt Nam,B2,NaN,75023K22B2196,30/11/2022,04/03/2023
66,62128,TRẦN HOÀI TRUNG,Nam,Việt Nam,B2,NaN,75023K22B2196,30/11/2022,04/03/2023
67,62129,ĐINH XUÂN TRƯỜNG,Nam,Việt Nam,B2,NaN,75023K22B2196,30/11/2022,04/03/2023


In [9]:
# prompt: column_info = {
#     "TT": ('int', "id"),
#     "Họ và tên": ('string', "name"),
#     "Giới\rtính": ('string', "gender"),
#     "Quốc tịch": ('string', "nationality"),
#     "Hạng": ('string', "class"),
#     "Số GPLX": ('string', "license_number"),
#     "Mã khóa học": ('string', "class_code"),
#     "Ngày khai\rgiảng": ("datetime64[D]", "start_date"),
#     "Ngày bế\rgiảng": ("datetime64[D]", "end_date")
# }
# format of datetime is "22/05/2020"
# from dfs, only accept these column in each pd, force cast the type of columns,  then combine them, treat the TT column as Index, then rename the columns, after all name the out as variable combined_dfs
# specific note to process pdf[0]:
# remove row 0,1
# row name isn't correct
# rename each columns follow the given name by index

column_info = {
    "TT": ('int', "id"),
    "Họ và tên": ('string', "name"),
    "Giới\rtính": ('string', "gender"),
    "Quốc tịch": ('string', "nationality"),
    "Hạng": ('string', "class"),
    "Số GPLX": ('string', "license_number"),
    "Mã khóa học": ('string', "class_code"),
    "Ngày khai\rgiảng": ("datetime64[D]", "start_date"),
    "Ngày bế\rgiảng": ("datetime64[D]", "end_date")
}

combined_dfs = []

# Process the first PDF (pdf[0])
if len(dfs) > 0:
    pdf = dfs[0].copy()
    pdf = pdf.iloc[2:]  # Remove rows 0 and 1
    pdf.columns = list(column_info.keys())[:len(pdf.columns)] # Rename columns by index
    combined_dfs.append(pdf)


for pdf in dfs[1:]:
    pdf = pdf[list(column_info.keys())]
    combined_dfs.append(pdf)

final_df_list = []
for pdf in combined_dfs:
  # Force cast the type of each column
  for col_name, (col_type, new_col_name) in column_info.items():
    if col_name in pdf.columns:
        if col_type == "datetime64[D]":
            try:
                pdf[col_name] = pd.to_datetime(pdf[col_name], format="%d/%m/%Y", errors='coerce')
            except ValueError:
                print("Invalid date format encountered.")

        elif col_type == "int":
            try:
              pdf[col_name] = pd.to_numeric(pdf[col_name], errors='coerce').astype('Int64') # Use nullable int type
            except ValueError:
              print(f"Error converting {col_name} to int")
        else:
            pdf[col_name] = pdf[col_name].astype(col_type)
  final_df_list.append(pdf)


combined_dfs = pd.concat(final_df_list)

# Set 'TT' as the index
combined_dfs = combined_dfs.set_index('TT')

# Rename columns
combined_dfs = combined_dfs.rename(columns={old_name: new_name for old_name, (_, new_name) in column_info.items() if old_name != 'TT'})


In [10]:
combined_dfs.to_csv('output.csv', index_label='no', index= True)


In [11]:
# prompt: show table columns type
# get top 100 row from combine dfs
# show the number of row  of combine dfs

print(combined_dfs.dtypes)
print(combined_dfs.head(100))
print(len(combined_dfs))


name              string[python]
gender            string[python]
nationality       string[python]
class             string[python]
license_number    string[python]
class_code        string[python]
start_date        datetime64[ns]
end_date          datetime64[ns]
dtype: object
                   name gender nationality class license_number  \
TT                                                                
1           TRẦN VĂN AN    Nam    Việt Nam     C           <NA>   
2            ĐÀO VĂN ÂN    Nam    Việt Nam     C   750209016017   
3       NGUYỄN HỒNG ANH    Nam    Việt Nam     C   790163275763   
4       LÊ TRẦN HỮU BẢO    Nam    Việt Nam     C           <NA>   
5          LÂM VĂN CHÂU    Nam    Việt Nam     C   770080004158   
..                  ...    ...         ...   ...            ...   
96   NGUYỄN HOÀNG DUYỆT    Nam    Việt Nam    B2   750208013091   
97          VŨ THANH HÀ    Nam    Việt Nam    B2   750215001306   
98            LÊ VĂN HÀ    Nam    Việt Nam    B2   7